<h1>Explaining the COVID-19 cases in Montreal</h1>

In [1]:
import numpy as np
import pandas as pd
import requests
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from bs4 import BeautifulSoup
from geopy.geocoders import Nominatim

import sys
if sys.version_info[0] < 3: 
    from StringIO import StringIO
else:
    from io import StringIO

## 1. Download and Explore Dataset

Get latest COVID cases from Sante Montreal's website.

Encoding is a little tricky due to special characters... from several tests, ***windows-1252*** fits best

In [2]:
url_cases = 'https://santemontreal.qc.ca/fileadmin/fichiers/Campagnes/coronavirus/situation-montreal/municipal.csv'
csv_cases = StringIO(requests.get(url_cases, headers={'user-agent': 'MTL Explorer'}).text)
df_cases = pd.read_csv(csv_cases, sep=";", encoding='cp1252')
df_cases.head()

,Arrondissement ou ville liée,Nombre de cas confirmés,Répartition des cas (%),Taux de cas pour 100 000 personnes,Nombre de décès,Taux de mortalité pour 100 000 personnes
0,Ahuntsic–Cartierville,2460,"8,6","1832,5",358,"266,7"
1,Anjou,729,"2,5","1703,4",51,"119,2"
2,Baie D'urfé,31,"0,1","*810,9",< 5,n.p.
3,Beaconsfield,62,"0,2","320,8",9,n.p.
4,Côte-des-Neiges–Notre-Dame-de-Grâce,2336,"8,1","1402,8",259,"155,5"


Data wrangling to match Montreal geojson

In [3]:
# Header names
df_cases.columns = ['Borough', 'ConfirmedCount', 'DistributionRate', 'ConfirmedPer100K', 'DeathCount', 'DeathPer100K']
# Clean up borough names
df_cases = df_cases[~df_cases["Borough"].isin(['L\'Île-Dorval', 'Territoire à confirmer', 'Total à Montréal'])]
df_cases['Borough'].replace('–', '-', regex=True, inplace=True)
df_cases['Borough'].replace('Baie D\'urfé', 'Baie-d\'Urfé', regex=True, inplace=True)
df_cases['Borough'].replace('Montréal Est', 'Montréal-Est', regex=True, inplace=True)
df_cases['Borough'].replace('Plateau Mont-Royal', 'Le Plateau-Mont-Royal', regex=True, inplace=True)
df_cases['Borough'].replace('Rosemont.*Patrie', 'Rosemont-La Petite-Patrie', regex=True, inplace=True)
df_cases['Borough'].replace('Sud-Ouest', 'Le Sud-Ouest', regex=True, inplace=True)
# Clean up noise
df_cases.replace('<[ ]?|\*', '', regex=True, inplace=True)
df_cases.replace('n\.p\.', '0', regex=True, inplace=True)
df_cases.replace(',', '.', regex=True, inplace=True)
df_cases.replace(to_replace=np.nan, value=0, inplace=True)
# Cast to correct data type
df_cases['ConfirmedCount'] = df_cases['ConfirmedCount'].astype('float')
df_cases['DistributionRate'] = df_cases['DistributionRate'].astype('float')
df_cases['ConfirmedPer100K'] = df_cases['ConfirmedPer100K'].astype('float')
df_cases['DeathCount'] = df_cases['DeathCount'].astype('float')
df_cases['DeathPer100K'] = df_cases['DeathPer100K'].astype('float')
# If no confirmed cases per 100k is not populated, assume it will be 100k
df_cases.loc[df_cases["ConfirmedPer100K"] <= 0.0, "ConfirmedPer100K"] = 100000
df_cases.loc[df_cases["DeathPer100K"] <= 0.0, "DeathPer100K"] = 100000
df_cases.head()

,Borough,ConfirmedCount,DistributionRate,ConfirmedPer100K,DeathCount,DeathPer100K
0,Ahuntsic-Cartierville,2460.0,8.6,1832.5,358.0,266.7
1,Anjou,729.0,2.5,1703.4,51.0,119.2
2,Baie-d'Urfé,31.0,0.1,810.9,5.0,100000.0
3,Beaconsfield,62.0,0.2,320.8,9.0,100000.0
4,Côte-des-Neiges-Notre-Dame-de-Grâce,2336.0,8.1,1402.8,259.0,155.5


Calculate the population from confirmed cases and confirmed per 100k.

This gives a very close approximation of the real numbers published in the last census from 2016.

In [4]:
df_cases['Population'] = round(df_cases['ConfirmedCount'] * 100000 / df_cases['ConfirmedPer100K'])
df_cases['Population'] = df_cases['Population'].astype(int)
# At the time of this writing, Senneville does not report ConfirmedPer100K
# Since its population cannot be estimated, we will use the data from 2016 Census
df_cases.loc[df_cases['Borough'] == 'Senneville','Population'] = 921
df_cases[['Borough','Population']].head(40)

,Borough,Population
0,Ahuntsic-Cartierville,134243
1,Anjou,42797
2,Baie-d'Urfé,3823
3,Beaconsfield,19327
4,Côte-des-Neiges-Notre-Dame-de-Grâce,166524
5,Côte-Saint-Luc,32448
6,Dollard-des-Ormeaux,48900
7,Dorval,18980
8,Hampstead,6973
9,Kirkland,20149


## 2. Visualize the data

Get geojson of all boroughs and cities

In [5]:
mtl_boro_url = 'http://donnees.ville.montreal.qc.ca/dataset/00bd85eb-23aa-4669-8f1b-ba9a000e3dd8/resource/e9b0f927-8f75-458c-8fda-b5da65cc8b73/download/limadmin.json'
mtl_boro_json = requests.get(mtl_boro_url).json()
mtl_boro_json['features'][0]['properties']

{'ABREV': 'OM',
 'AIRE': 3813355.72326504,
 'CODEID': '11',
 'CODEMAMROT': 'REM05',
 'MUNID': 66023,
 'NOM': 'Outremont',
 'NUM': 5,
 'PERIM': 10836.6706340882,
 'TYPE': 'Arrondissement'}

Extract area information (in km<sup>2</sup>) and translate

In [6]:
df_boro_area = pd.json_normalize(mtl_boro_json['features'])
df_boro_area = df_boro_area.loc[:,['properties.NOM','properties.AIRE', 'properties.TYPE']]
df_boro_area.columns = ['Borough', 'Area', 'BoroughType']
df_boro_area['Area'] = df_boro_area['Area'] / 1000000
df_boro_area.loc[df_boro_area["BoroughType"] == 'Arrondissement', "BoroughType"] = 0
df_boro_area.loc[df_boro_area["BoroughType"] == 'Ville liée', "BoroughType"] = 1
df_boro_area['BoroughType'] = df_boro_area['BoroughType'].astype(int)
df_boro_area.head()

,Borough,Area,BoroughType
0,Outremont,3.813356,0
1,LaSalle,25.197268,0
2,Mont-Royal,7.445560,1
3,Ville-Marie,21.500632,0
4,Le Plateau-Mont-Royal,8.151665,0


Left join the above to our main dataset

In [7]:
df_cases = df_cases.merge(right=df_boro_area, how='left', on='Borough')
df_cases.head()

,Borough,ConfirmedCount,DistributionRate,ConfirmedPer100K,DeathCount,DeathPer100K,Population,Area,BoroughType
0,Ahuntsic-Cartierville,2460.0,8.6,1832.5,358.0,266.7,134243,25.571187,0
1,Anjou,729.0,2.5,1703.4,51.0,119.2,42797,13.878194,0
2,Baie-d'Urfé,31.0,0.1,810.9,5.0,100000.0,3823,8.025921,1
3,Beaconsfield,62.0,0.2,320.8,9.0,100000.0,19327,24.922506,1
4,Côte-des-Neiges-Notre-Dame-de-Grâce,2336.0,8.1,1402.8,259.0,155.5,166524,21.483755,0


Calculate each borough's population density (per km<sup>2</sup>)

In [8]:
df_cases['Density'] = df_cases['Population'] / df_cases['Area']
df_cases[['Borough', 'Population', 'Area', 'Density']].head()

,Borough,Population,Area,Density
0,Ahuntsic-Cartierville,134243,25.571187,5249.775752
1,Anjou,42797,13.878194,3083.758656
2,Baie-d'Urfé,3823,8.025921,476.331652
3,Beaconsfield,19327,24.922506,775.483821
4,Côte-des-Neiges-Notre-Dame-de-Grâce,166524,21.483755,7751.159068


Get Montreal's coordinates

In [9]:
address = 'Montreal, Quebec, Canada'

geolocator = Nominatim(user_agent="MTL Explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Montreal are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Montreal are 45.4972159, -73.6103642.


Visualize cases on the Montreal island and ensure that the choropleth properly matches the names of our dataframe. Note that the independent suburbs are highlighted to distinguish them from the city's municipalities.

In [10]:
mtl_map = folium.Map(location=[latitude,longitude], zoom_start=10, tiles='OpenStreetMap')

# Function to style suburbs not part of te cirty of Montreal
boro_style = lambda x: {
    'stroke': True if x['properties']['TYPE'] == 'Ville liée' else False,
    'weight': 1.5,
    'color': 'purple',
    'fillOpacity': 0
}
suburb_contours = folium.features.GeoJson(mtl_boro_json, style_function=boro_style)

# Counts of confirmed cases
choropleth = folium.Choropleth(
    mtl_boro_json,
    data=df_cases,
    columns=['Borough', 'ConfirmedCount'],
    key_on='feature.properties.NOM',
    fill_color='YlOrRd',
    fill_opacity=0.7, 
    line_opacity=0.2,
    legend_name='COVID cases'
).add_to(mtl_map)

mtl_map.add_child(suburb_contours)
mtl_map

Interesting that not many cases are recorded on the West Island (basically, west of Lachine/Saint-Laurent). Can it be due to population density in those boroughs?

In [11]:
mtl_map = folium.Map(location=[latitude,longitude], zoom_start=10, tiles='OpenStreetMap')

# Densities by borough
choropleth = folium.Choropleth(
    mtl_boro_json,
    data=df_cases,
    columns=['Borough', 'Density'],
    key_on='feature.properties.NOM',
    fill_color='YlGn', 
    fill_opacity=0.7, 
    line_opacity=0.2,
    legend_name='Population Density'
).add_to(mtl_map)

mtl_map.add_child(suburb_contours)
mtl_map

Population density parially answers the above question. At first glance, it seems as though the independent suburbs (highlighted with bold contour) are doing better than the city's municipalities. However, the Plateau, which is the densest area on the map, has not had that many cases compared to neighboring municipalities.

Calculate Latitude/Longitude of each borough based on its geojson coordinates

In [41]:
df_cases['Latitude'] = 0.0
df_cases['Longitude'] = 0.0
boros = mtl_boro_json['features']

# Loop over all coordinates of each borough and record min/max
for idx in range(len(boros)):
    coords = boros[idx]['geometry']['coordinates'][0][0]
    ll_min = [180.0, 180.0]
    ll_max = [-180.0, -180.0]
    for pnt in range(len(coords)):
        ll_min = list(map(min, zip(ll_min, coords[pnt])))
        ll_max = list(map(max, zip(ll_max, coords[pnt])))
    # Approximate the borough center by taking the average extremes in each direction
    df_cases.loc[df_cases['Borough'] == boros[idx]['properties']['NOM'], 'Latitude'] = (ll_max[1]+ll_min[1])/2
    df_cases.loc[df_cases['Borough'] == boros[idx]['properties']['NOM'], 'Longitude'] = (ll_max[0]+ll_min[0])/2

df_cases.head()

,Borough,ConfirmedCount,DistributionRate,ConfirmedPer100K,DeathCount,DeathPer100K,Population,Area,BoroughType,Density,Latitude,Longitude
0,Ahuntsic-Cartierville,2460.0,8.6,1832.5,358.0,266.7,134243,25.571187,0,5249.775752,45.545058,-73.692788
1,Anjou,729.0,2.5,1703.4,51.0,119.2,42797,13.878194,0,3083.758656,45.612252,-73.569294
2,Baie-d'Urfé,31.0,0.1,810.9,5.0,100000.0,3823,8.025921,1,476.331652,45.416696,-73.914343
3,Beaconsfield,62.0,0.2,320.8,9.0,100000.0,19327,24.922506,1,775.483821,45.415412,-73.857932
4,Côte-des-Neiges-Notre-Dame-de-Grâce,2336.0,8.1,1402.8,259.0,155.5,166524,21.483755,0,7751.159068,45.483677,-73.635721


## 3. Search for Healthcare Institutions

Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.

In [13]:
CLIENT_ID = 'BQFGSANCVA4JLVSFDADVHZHJRMA2INX4URMRIFHJ0QGHRVPV' # your Foursquare ID
CLIENT_SECRET = 'TR00D4NNSNOSWX3JK1BZAOBFSQN3EVRD1BYXSCANUP3DRSXH' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentials:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentials:
CLIENT_ID: BQFGSANCVA4JLVSFDADVHZHJRMA2INX4URMRIFHJ0QGHRVPV
CLIENT_SECRET:TR00D4NNSNOSWX3JK1BZAOBFSQN3EVRD1BYXSCANUP3DRSXH


Get the neighborhood's name.

In [14]:
df_cases.loc[0, 'Borough']

'Ahuntsic-Cartierville'

Get the neighborhood's latitude and longitude values.

In [15]:
borough_latitude = df_cases.loc[0, 'Latitude'] # Borough latitude value
borough_longitude = df_cases.loc[0, 'Longitude'] # Borough longitude value
borough_name = df_cases.loc[0, 'Borough'] # Borough name

print('Latitude and longitude values of {} are {}, {}.'.format(borough_name, 
                                                               borough_latitude, 
                                                               borough_longitude))

Latitude and longitude values of Ahuntsic-Cartierville are 45.545057647731554, -73.6927883383282.


First, let's create the GET request URL.

In [16]:
# type your answer here
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    borough_latitude, 
    borough_longitude, 
    radius, 
    LIMIT)
url


'https://api.foursquare.com/v2/venues/explore?&client_id=BQFGSANCVA4JLVSFDADVHZHJRMA2INX4URMRIFHJ0QGHRVPV&client_secret=TR00D4NNSNOSWX3JK1BZAOBFSQN3EVRD1BYXSCANUP3DRSXH&v=20180605&ll=45.545057647731554,-73.6927883383282&radius=500&limit=100'

Send the GET request and examine the resutls

In [17]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f382b8a99578521cc39cc6a'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-4c24e9dcf1272d7f82d783c5-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/park_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d163941735',
         'name': 'Park',
         'pluralName': 'Parks',
         'primary': True,
         'shortName': 'Park'}],
       'id': '4c24e9dcf1272d7f82d783c5',
       'location': {'address': '908, boul. Gouin O.',
        'cc': 'CA',
        'city': 'Montréal',
        'country': 'Canada',
        'distance': 184,
        'formattedAddress': ['908, boul. Gouin O.', 'Montréal QC', 'Canada'],
        'labeledLatLngs': [{'label': 'display',
          'lat': 45.54667364344165,
          'lng': -73.69329

We define function ***getNearbyVenues***, which is very similar to that from the lab, but also allows us to parse venues constrained by a set of categories and for varying radii

In [35]:
def getNearbyVenues(names, latitudes, longitudes, radii, categories):
    print("Querying Foursquare API...")
    venues = []
    
    for name, lat, lon, radius in zip(names, latitudes, longitudes, radii):
        print(name)
        
        url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categoryId={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        lat, 
        lon, 
        radius, 
        LIMIT,
        ','.join(categories))
    
        results = requests.get(url).json()
        
        for item in results['response']['venues']:
            if (item['categories'][0]['id'] in categories):
                venue = {
                    'Borough': name,
                    'Name': item['name'],
                    'Category': item['categories'][0]['name'],
                    'Latitude': item['location']['lat'],
                    'Longitude': item['location']['lng']
                }
                venues.append(venue)

    return(pd.DataFrame(venues))

Let us look at the availability of healthcare centers in each borough. Since Foursquare needs a radius around the point-of-interest (POI), we calculate it by converting the borough's area (in KM<sup>2</sup>). In some cases, a POI could be referenced by more than one borough if ever their radii overlap.

In [19]:
categories = ['4bf58dd8d48988d104941735', # Medical Center
              '4bf58dd8d48988d194941735', # Emergency Room
              '4bf58dd8d48988d196941735', # Hospital
              '58daa1558bbb0b01f18ec1f7', # Hospital Ward
              '56aa371be4b08b9a8d573526'] # Urgent Care Center

venues = getNearbyVenues(names=df_cases['Borough'],
                latitudes=df_cases['Latitude'],
                longitudes=df_cases['Longitude'],
                radii=df_cases['Area'] ** (1/2) * 1000 * 0.75,
                categories=categories)

Querying Foursquare API...
Ahuntsic-Cartierville
Anjou
Baie-d'Urfé
Beaconsfield
Côte-des-Neiges-Notre-Dame-de-Grâce
Côte-Saint-Luc
Dollard-des-Ormeaux
Dorval
Hampstead
Kirkland
Lachine
LaSalle
L'Île-Bizard-Sainte-Geneviève
Mercier-Hochelaga-Maisonneuve
Montréal-Est
Montréal-Nord
Montréal-Ouest
Mont-Royal
Outremont
Pierrefonds-Roxboro
Le Plateau-Mont-Royal
Pointe-Claire
Rivière-des-Prairies-Pointe-aux-Trembles
Rosemont-La Petite-Patrie
Sainte-Anne-de-Bellevue
Saint-Laurent
Saint-Léonard
Senneville
Le Sud-Ouest
Verdun
Ville-Marie
Villeray-Saint-Michel-Parc-Extension
Westmount


Let's check the size of the resulting dataframe

In [20]:
print(venues.shape)
venues.head()

(371, 5)


,Borough,Name,Category,Latitude,Longitude
0,Ahuntsic-Cartierville,Centre d'hébergement Notre-Dame-de-la-Merci,Hospital,45.547159,-73.687192
1,Ahuntsic-Cartierville,Hôpital du Sacré-Coeur de Montréal,Hospital,45.532732,-73.714517
2,Ahuntsic-Cartierville,CSSS de Laval,Medical Center,45.544243,-73.739664
3,Ahuntsic-Cartierville,Centre De Psychologie Infantile,Medical Center,45.551163,-73.675667
4,Ahuntsic-Cartierville,Cliniques Zéro Gravité,Medical Center,45.566957,-73.751206


Let's check how many venues were returned for each neighborhood

In [21]:
venues.groupby('Borough').count()

,Name,Category,Latitude,Longitude
Borough,,,,
Ahuntsic-Cartierville,17,17,17,17
Anjou,10,10,10,10
Beaconsfield,8,8,8,8
Côte-Saint-Luc,11,11,11,11
Côte-des-Neiges-Notre-Dame-de-Grâce,21,21,21,21
Dollard-des-Ormeaux,9,9,9,9
Dorval,4,4,4,4
Hampstead,9,9,9,9
Kirkland,8,8,8,8


Let us plot the healthcare center locations vs mortality rate per borough.

In [22]:
mtl_map = folium.Map(location=[latitude,longitude], zoom_start=10, tiles='OpenStreetMap')

# Densities by borough
choropleth = folium.Choropleth(
    mtl_boro_json,
    data=df_cases,
    columns=['Borough', 'DeathCount'],
    key_on='feature.properties.NOM',
    fill_color='YlOrRd', 
    fill_opacity=0.7, 
    line_opacity=0.2,
).add_to(mtl_map)

for name, lat, lon, category in zip(venues['Name'],venues['Latitude'],venues['Longitude'],venues['Category']):
    info = name + "\n(" + category +")"
    label = folium.Popup(info, parse_html=True)
    folium.Circle(
        [lat, lon],
        radius=100,
        popup=label,
        fill=True,
        fill_color='Or',
        fill_opacity=0.2).add_to(mtl_map)

mtl_map

We can observe that the mortality rate is quite high despite the availability of medical institutions.
Now let us plot these same healthcare facilities and color code the boroughs based on how many are within their radius. 

In [37]:
venue_count = venues[['Borough', 'Category']].groupby('Borough').count()
venue_count = venue_count.reset_index()
venue_count.columns = ['Borough', 'Count']
venue_count.head()

,Borough,Count
0,Ahuntsic-Cartierville,17
1,Anjou,10
2,Beaconsfield,8
3,Côte-Saint-Luc,11
4,Côte-des-Neiges-Notre-Dame-de-Grâce,21


In [40]:
mtl_map = folium.Map(location=[latitude,longitude], zoom_start=10, tiles='OpenStreetMap')

# Densities by borough
choropleth = folium.Choropleth(
    mtl_boro_json,
    data=venue_count,
    columns=['Borough', 'Count'],
    key_on='feature.properties.NOM',
    fill_color='YlOrRd', 
    fill_opacity=0.7, 
    nan_fill_color='Pink',
    nan_fill_opacity=0.7,
    line_opacity=0.2,
).add_to(mtl_map)

for lat, lon, boro, area in zip(df_cases['Latitude'], df_cases['Longitude'], df_cases['Borough'], df_cases['Area']):
    folium.Circle(
        [lat, lon],
        radius=area ** (1/2) * 1000 * 0.75,
        tooltip=boro,
        fill=True,
        weight=1,
        fill_color='Or',
        fill_opacity=0.0).add_to(mtl_map)

for name, lat, lon, category in zip(venues['Name'],venues['Latitude'],venues['Longitude'],venues['Category']):
    info = name + "\n(" + category +")"
    label = folium.Popup(info, parse_html=True)
    folium.Circle(
        [lat, lon],
        radius=100,
        popup=label,
        fill=True,
        fill_color='Or',
        fill_opacity=0.2).add_to(mtl_map)

mtl_map

## 4. Analyze Each Borough

In [24]:
# one hot encoding
montreal_onehot = pd.get_dummies(venues[['Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
montreal_onehot['Borough'] = venues['Borough'] 

# move neighborhood column to the first column
fixed_columns = [montreal_onehot.columns[-1]] + list(montreal_onehot.columns[:-1])
montreal_onehot = montreal_onehot[fixed_columns]

montreal_onehot.head()

,Borough,Emergency Room,Hospital,Medical Center
0,Ahuntsic-Cartierville,0,1,0
1,Ahuntsic-Cartierville,0,1,0
2,Ahuntsic-Cartierville,0,0,1
3,Ahuntsic-Cartierville,0,0,1
4,Ahuntsic-Cartierville,0,0,1


And let's examine the new dataframe size.

In [25]:
montreal_onehot.shape

(371, 4)

#### Next, let's group rows by borough and by taking the sum of each category

In [26]:
montreal_grouped = montreal_onehot.groupby('Borough').sum().reset_index()
montreal_grouped

,Borough,Emergency Room,Hospital,Medical Center
0,Ahuntsic-Cartierville,1,2,14
1,Anjou,0,0,10
2,Beaconsfield,1,1,6
3,Côte-Saint-Luc,0,5,6
4,Côte-des-Neiges-Notre-Dame-de-Grâce,0,11,10
5,Dollard-des-Ormeaux,0,0,9
6,Dorval,0,0,4
7,Hampstead,0,1,8
8,Kirkland,1,0,7
9,L'Île-Bizard-Sainte-Geneviève,0,1,4


#### Calculate the patient-per-clinic ratios

In [27]:
montreal_grouped = df_cases.merge(right=montreal_grouped, how='left', on='Borough')
montreal_grouped.replace(to_replace=np.nan, value=0, inplace=True)
montreal_grouped['PatientsPerClinic'] = montreal_grouped['Population'] / (montreal_grouped['Emergency Room'] + montreal_grouped['Hospital'] + montreal_grouped['Medical Center'])
montreal_grouped.replace(to_replace=np.inf, value=0, inplace=True)
montreal_grouped.head()

,Borough,ConfirmedCount,DistributionRate,ConfirmedPer100K,DeathCount,DeathPer100K,Population,Area,BoroughType,Density,Latitude,Longitude,Emergency Room,Hospital,Medical Center,PatientsPerClinic
0,Ahuntsic-Cartierville,2460.0,8.6,1832.5,358.0,266.7,134243,25.571187,0,5249.775752,45.545058,-73.692788,1.0,2.0,14.0,7896.647059
1,Anjou,729.0,2.5,1703.4,51.0,119.2,42797,13.878194,0,3083.758656,45.612252,-73.569294,0.0,0.0,10.0,4279.700000
2,Baie-d'Urfé,31.0,0.1,810.9,5.0,100000.0,3823,8.025921,1,476.331652,45.416696,-73.914343,0.0,0.0,0.0,0.000000
3,Beaconsfield,62.0,0.2,320.8,9.0,100000.0,19327,24.922506,1,775.483821,45.415412,-73.857932,1.0,1.0,6.0,2415.875000
4,Côte-des-Neiges-Notre-Dame-de-Grâce,2336.0,8.1,1402.8,259.0,155.5,166524,21.483755,0,7751.159068,45.483677,-73.635721,0.0,11.0,10.0,7929.714286


## 5. Cluster Boroughs

Run *k*-means to cluster the neighborhood into 5 clusters.

In [42]:
# set number of clusters
kclusters = 3

montreal_grouped_clustering = montreal_grouped[
    ['ConfirmedCount',
     'DeathCount',
     'BoroughType',
     'Density',
     'Emergency Room',
     'Hospital',
     'Medical Center',
     'PatientsPerClinic']
]
montreal_grouped_clustering = StandardScaler().fit_transform(montreal_grouped_clustering)

# run k-means clustering
kmeans = KMeans(init="k-means++", n_clusters=kclusters, random_state=42, n_init=12)
kmeans.fit(montreal_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 0, 2, 2, 1, 2, 2, 2, 2, 2], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [29]:
montreal_merged = df_cases

print(montreal_merged.shape)
print(montreal_grouped_clustering.shape)

# merge montreal_grouped with montreal_data to add latitude/longitude for each neighborhood
#montreal_merged = montreal_merged.join(neighborhoods_venues_sorted.set_index('Borough'), on='Borough')
montreal_grouped['Cluster Labels'] = kmeans.labels_

montreal_grouped.head() # check the last columns!

(33, 12)
(33, 8)


,Borough,ConfirmedCount,DistributionRate,ConfirmedPer100K,DeathCount,DeathPer100K,Population,Area,BoroughType,Density,Latitude,Longitude,Emergency Room,Hospital,Medical Center,PatientsPerClinic,Cluster Labels
0,Ahuntsic-Cartierville,2460.0,8.6,1832.5,358.0,266.7,134243,25.571187,0,5249.775752,45.545058,-73.692788,1.0,2.0,14.0,7896.647059,1
1,Anjou,729.0,2.5,1703.4,51.0,119.2,42797,13.878194,0,3083.758656,45.612252,-73.569294,0.0,0.0,10.0,4279.700000,0
2,Baie-d'Urfé,31.0,0.1,810.9,5.0,100000.0,3823,8.025921,1,476.331652,45.416696,-73.914343,0.0,0.0,0.0,0.000000,2
3,Beaconsfield,62.0,0.2,320.8,9.0,100000.0,19327,24.922506,1,775.483821,45.415412,-73.857932,1.0,1.0,6.0,2415.875000,2
4,Côte-des-Neiges-Notre-Dame-de-Grâce,2336.0,8.1,1402.8,259.0,155.5,166524,21.483755,0,7751.159068,45.483677,-73.635721,0.0,11.0,10.0,7929.714286,1


Finally, let's visualize the resulting clusters

In [30]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(montreal_grouped['Latitude'], montreal_grouped['Longitude'], montreal_grouped['Borough'], montreal_grouped['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 6. Examine Clusters

Examine each cluster and determine the discriminating categories that distinguish each cluster.

#### Cluster 1: Boroughs to watch out for. These boroughs seem to be under control but exhibit a moderate-to-relatively-high patient per clinic ratio. This could become a bottleneck for testing and patient care if cases start going up all of the sudden. The presence of boroughs with low cases like Mont-Royal, Outremont and Westmount could be explained by their location being surrounded by boroughs with higher confirmed/deatch counts, which makes them be at risk.

In [31]:
montreal_grouped.loc[montreal_grouped['Cluster Labels'] == 0]

,Borough,ConfirmedCount,DistributionRate,ConfirmedPer100K,DeathCount,DeathPer100K,Population,Area,BoroughType,Density,Latitude,Longitude,Emergency Room,Hospital,Medical Center,PatientsPerClinic,Cluster Labels
1,Anjou,729.0,2.5,1703.4,51.0,119.2,42797,13.878194,0,3083.758656,45.612252,-73.569294,0.0,0.0,10.0,4279.700000,0
10,Lachine,693.0,2.4,1557.7,99.0,222.5,44489,23.127786,0,1923.616900,45.447692,-73.676965,0.0,5.0,11.0,2780.562500,0
11,LaSalle,1281.0,4.5,1666.8,181.0,235.5,76854,25.197268,0,3050.092595,45.433673,-73.602981,0.0,7.0,6.0,5911.846154,0
17,Mont-Royal,281.0,1.0,1385.9,75.0,369.9,20276,7.445560,1,2723.233697,45.507204,-73.653306,0.0,4.0,13.0,1192.705882,0
18,Outremont,280.0,1.0,1168.9,11.0,45.9,23954,3.813356,0,6281.606474,45.515919,-73.608464,0.0,0.0,14.0,1711.000000,0
19,Pierrefonds-Roxboro,579.0,2.0,835.5,36.0,52.0,69300,33.765273,0,2052.404527,45.479656,-73.863018,0.0,1.0,12.0,5330.769231,0
20,Le Plateau-Mont-Royal,1058.0,3.7,1017.3,139.0,133.7,104001,8.151665,0,12758.252329,45.523276,-73.585827,0.0,5.0,12.0,6117.705882,0
25,Saint-Laurent,1146.0,4.0,1159.6,120.0,121.4,98827,43.077847,0,2294.148983,45.496464,-73.712476,1.0,4.0,9.0,7059.071429,0
26,Saint-Léonard,1178.0,4.1,1504.4,62.0,79.2,78304,13.550689,0,5778.598960,45.588768,-73.596788,0.0,3.0,9.0,6525.333333,0
28,Le Sud-Ouest,1014.0,3.5,1297.5,181.0,231.6,78150,18.144269,0,4307.145020,45.467900,-73.578286,0.0,8.0,8.0,4884.375000,0


#### Cluster 2: Very affected boroughs. High confirmed cases, likely due to population density and high patient per clinic ratio.

In [32]:
montreal_grouped.loc[montreal_grouped['Cluster Labels'] == 1]

,Borough,ConfirmedCount,DistributionRate,ConfirmedPer100K,DeathCount,DeathPer100K,Population,Area,BoroughType,Density,Latitude,Longitude,Emergency Room,Hospital,Medical Center,PatientsPerClinic,Cluster Labels
0,Ahuntsic-Cartierville,2460.0,8.6,1832.5,358.0,266.7,134243,25.571187,0,5249.775752,45.545058,-73.692788,1.0,2.0,14.0,7896.647059,1
4,Côte-des-Neiges-Notre-Dame-de-Grâce,2336.0,8.1,1402.8,259.0,155.5,166524,21.483755,0,7751.159068,45.483677,-73.635721,0.0,11.0,10.0,7929.714286,1
13,Mercier-Hochelaga-Maisonneuve,2463.0,8.6,1810.7,374.0,275.0,136025,27.408412,0,4962.892438,45.573555,-73.536803,0.0,4.0,11.0,9068.333333,1
15,Montréal-Nord,2639.0,9.2,3132.9,249.0,295.6,84235,12.430208,0,6776.636277,45.604793,-73.630069,0.0,3.0,5.0,10529.375000,1
22,Rivière-des-Prairies-Pointe-aux-Trembles,2227.0,7.8,2086.3,240.0,224.8,106744,50.047004,0,2132.874907,45.660083,-73.549307,0.0,4.0,7.0,9704.000000,1
23,Rosemont-La Petite-Patrie,1739.0,6.1,1245.8,237.0,169.8,139589,15.886530,0,8786.626003,45.553784,-73.585250,1.0,4.0,10.0,9305.933333,1
31,Villeray-Saint-Michel-Parc-Extension,2339.0,8.1,1626.0,132.0,91.8,143850,16.477356,0,8730.162703,45.554407,-73.617423,2.0,3.0,12.0,8461.764706,1


#### Cluster 3: Safe zones, low confirmed cases and deaths. Usually, high ratio of number of patients to  clinic availability

In [33]:
montreal_grouped.loc[montreal_grouped['Cluster Labels'] == 2]

,Borough,ConfirmedCount,DistributionRate,ConfirmedPer100K,DeathCount,DeathPer100K,Population,Area,BoroughType,Density,Latitude,Longitude,Emergency Room,Hospital,Medical Center,PatientsPerClinic,Cluster Labels
2,Baie-d'Urfé,31.0,0.1,810.9,5.0,100000.0,3823,8.025921,1,476.331652,45.416696,-73.914343,0.0,0.0,0.0,0.000000,2
3,Beaconsfield,62.0,0.2,320.8,9.0,100000.0,19327,24.922506,1,775.483821,45.415412,-73.857932,1.0,1.0,6.0,2415.875000,2
5,Côte-Saint-Luc,541.0,1.9,1667.3,62.0,191.1,32448,6.810210,1,4764.611012,45.468246,-73.665812,0.0,5.0,6.0,2949.818182,2
6,Dollard-des-Ormeaux,440.0,1.5,899.8,81.0,165.6,48900,15.065159,1,3245.900081,45.484486,-73.812876,0.0,0.0,9.0,5433.333333,2
7,Dorval,205.0,0.7,1080.1,50.0,263.4,18980,28.156150,1,674.097852,45.456708,-73.751426,0.0,0.0,4.0,4745.000000,2
8,Hampstead,59.0,0.2,846.1,5.0,100000.0,6973,1.768055,1,3943.881204,45.481475,-73.642963,0.0,1.0,8.0,774.777778,2
9,Kirkland,124.0,0.4,615.4,23.0,114.1,20149,9.687581,1,2079.879321,45.452854,-73.868841,1.0,0.0,7.0,2518.625000,2
12,L'Île-Bizard-Sainte-Geneviève,223.0,0.8,1211.1,45.0,244.4,18413,36.532506,0,504.016887,45.494875,-73.913765,0.0,1.0,4.0,3682.600000,2
14,Montréal-Est,46.0,0.2,1194.8,5.0,100000.0,3850,13.974007,1,275.511519,45.628823,-73.526228,0.0,2.0,7.0,427.777778,2
16,Montréal-Ouest,29.0,0.1,574.3,5.0,100000.0,5050,1.419449,1,3557.717742,45.453110,-73.650067,0.0,0.0,2.0,2525.000000,2
